# Part 1: Credit Card Feature Engineering

In this tutorial, you'll learn how to create features using the [Kaggle Credit Card Default dataset](./data/UCI_Credit_Card.csv). Start by installing the necessary dependencies below. 

In [10]:
pip install pandas numpy scikit-learn


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Setup: Load Data and Libraries

We first need to install the necessary packages, load in the data, and verify that it has been accessed correctly. This can be done by running the code below.

In [65]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the credit card default dataset
df = pd.read_csv('data/UCI_Credit_Card.csv')

# Display original features
print("Original features (first 5 rows):")
print(df[['LIMIT_BAL', 'BILL_AMT1', 'PAY_AMT1', 'AGE']].head())

Original features (first 5 rows):
   LIMIT_BAL  BILL_AMT1  PAY_AMT1  AGE
0    20000.0     3913.0       0.0   24
1   120000.0     2682.0       0.0   26
2    90000.0    29239.0    1518.0   34
3    50000.0    46990.0    2000.0   37
4    50000.0     8617.0    2000.0   57


### Interaction Type 1: Division (Ratios)

Ratios capture relative relationships between features. Often, these are the most useful interaction features for predictive tasks. The code below determines:
1. **Utilization Rate**: How much of the credit limit is being used
2. **Payment Rate**: How much of the bill was paid

In [57]:
# Utilization rate: How much of credit limit is being used
df['utilization_rate'] = df['BILL_AMT1'] / (df['LIMIT_BAL'] + 1)  # +1 avoids division by zero

# Payment rate: How much of the bill was paid
df['payment_rate'] = df['PAY_AMT1'] / (df['BILL_AMT1'] + 1)

# Replace any infinity values with 0
df['utilization_rate'] = df['utilization_rate'].replace([np.inf, -np.inf], 0)
df['payment_rate'] = df['payment_rate'].replace([np.inf, -np.inf], 0)

print("\nRatio features (first 5 rows):")
print(df[['utilization_rate', 'payment_rate']].head())


Ratio features (first 5 rows):
   utilization_rate  payment_rate
0          0.195640      0.000000
1          0.022350      0.000000
2          0.324874      0.051915
3          0.939781      0.042561
4          0.172337      0.232072


### Interaction Type 2: Subtraction (Differences)
Differences show gaps or remaining capacity. For our credit data, we can use subtraction to determine:
1. **Available Credit**: How much credit a person still has available.
2. **Underpayment**: How much of the bill wasn't paid

In [58]:
# Available credit: How much credit is still available
df['available_credit'] = df['LIMIT_BAL'] - df['BILL_AMT1']

# Underpayment: How much of the bill wasn't paid
df['underpayment'] = df['BILL_AMT1'] - df['PAY_AMT1']

**How to Handle Infinite Values?**
Sometimes division creates infinity (when dividing by very small numbers). This will cause major issues, so we can replace these with 0 so the model can handle them.

In [59]:
# Replace any infinity values with 0
df['utilization_rate'] = df['utilization_rate'].replace([np.inf, -np.inf], 0)
df['payment_rate'] = df['payment_rate'].replace([np.inf, -np.inf], 0)

print("\nRatio features (first 5 rows):")
print(df[['utilization_rate', 'payment_rate']].head())


Ratio features (first 5 rows):
   utilization_rate  payment_rate
0          0.195640      0.000000
1          0.022350      0.000000
2          0.324874      0.051915
3          0.939781      0.042561
4          0.172337      0.232072


### Using Your Engineered Features
Once you've created interaction features, add them to your model:

In [60]:
# Select features including engineered ones
feature_columns = [
    # Original features
    'LIMIT_BAL', 'AGE', 'PAY_0', 'BILL_AMT1', 'PAY_AMT1',
    # Engineered interaction features
    'utilization_rate', 'payment_rate', 'available_credit', 'underpayment'
]

# Prepare X and y
X = df[feature_columns]
y = df['default.payment.next.month']

# Handle any missing values
X = X.fillna(X.median())

# Split and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
predictions = model.predict(X_test)
print(f"\nModel Accuracy: {accuracy_score(y_test, predictions):.4f}")


Model Accuracy: 0.7947


### One-Hot Encoding in Python
One-hot encoding creates a new binary (0/1) column for each category. If a row belongs to that category, it gets a 1. Otherwise, it gets a 0. We have education data that is encoded with numbers mapping to types of education:
- 1: Graduate school
- 2: University
- 3: High school
- 4: Others
- 5: Unknown
- 6: Unknown

In [43]:
# Look at a categorical feature
print("Original EDUCATION values:")
print(df['EDUCATION'].value_counts())

Original EDUCATION values:
EDUCATION
2    14030
1    10585
3     4917
5      280
4      123
6       51
0       14
Name: count, dtype: int64


This can be converted using `pd.get_dummies` to create dummy variables for education.

In [42]:
# Create dummy variables for EDUCATION
education_encoded = pd.get_dummies(df['EDUCATION'], prefix='Education', dtype=int)

print("\nEncoded columns:")
print(education_encoded.head())


Encoded columns:
   Education_0  Education_1  Education_2  Education_3  Education_4  \
0            0            0            1            0            0   
1            0            0            1            0            0   
2            0            0            1            0            0   
3            0            0            1            0            0   
4            0            0            1            0            0   

   Education_5  Education_6  
0            0            0  
1            0            0  
2            0            0  
3            0            0  
4            0            0  


The original categorical column can be dropped and these new encoded columns can be added.

In [ ]:
# Drop the original categorical column
df_encoded = df.drop('EDUCATION', axis=1)

# Add the encoded columns
df_encoded = pd.concat([df_encoded, education_encoded], axis=1)

print(f"\nOriginal columns: {df.shape[1]}")
print(f"After encoding: {df_encoded.shape[1]}")


Original columns: 25
After encoding: 31


### Creating Boolean Features
Instead of encoding all categories, you can create boolean (True/False) features for specific conditions that matter for predictions. For example, we can encode if they are in a high-risk education category or if they are a graduate student.

In [47]:
# Create boolean: Is education unknown?
df['has_unknown_education'] = (df['EDUCATION'].isin([5, 6])).astype(int)

# Create boolean: Is this person a graduate?
df['is_graduate'] = (df['EDUCATION'] == 1).astype(int)

# Create boolean: High credit limit (over $100k)?
df['has_high_limit'] = (df['LIMIT_BAL'] > 100000).astype(int)

print("\nBoolean features (first 10 rows):")
print(df[['EDUCATION', 'LIMIT_BAL', 'has_unknown_education', 'is_graduate', 'has_high_limit']].head(10))


Boolean features (first 10 rows):
   EDUCATION  LIMIT_BAL  has_unknown_education  is_graduate  has_high_limit
0          2    20000.0                      0            0               0
1          2   120000.0                      0            0               1
2          2    90000.0                      0            0               0
3          2    50000.0                      0            0               0
4          2    50000.0                      0            0               0
5          1    50000.0                      0            1               0
6          1   500000.0                      0            1               1
7          2   100000.0                      0            0               0
8          3   140000.0                      0            0               1
9          3    20000.0                      0            0               0


### Using Encoded Features in Your Model
Once the features have been defined, you can then update the model to include these features in addition to the raw dataset. We can add these boolean features to our engineered interactions and the original features.

In [63]:
# Create boolean features
df['is_graduate'] = (df['EDUCATION'] == 1).astype(int)
df['has_high_limit'] = (df['LIMIT_BAL'] > 100000).astype(int)
df['has_high_balance'] = (df['BILL_AMT1'] > 50000).astype(int)

# Select ALL features: original + engineered ratios/differences + boolean
feature_columns = [
    # Original features
    'LIMIT_BAL', 'AGE', 'PAY_0', 'BILL_AMT1', 'PAY_AMT1',
    # Engineered interaction features
    'utilization_rate', 'payment_rate', 'available_credit', 'underpayment',
    # Boolean features
    'is_graduate', 'has_high_limit', 'has_high_balance'
]

X = df[feature_columns]
y = df['default.payment.next.month']

# Handle missing values
X = X.fillna(X.median())

# Train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
predictions = model.predict(X_test)
print(f"\nModel Accuracy: {accuracy_score(y_test, predictions):.4f}")


Model Accuracy: 0.7982
